## Decision Tree
titanic Dtaset

In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('train.csv')

In [4]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# surviveed is y
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 66.2+ KB


- Age has NaN for Many as we have 714 enteries only.
    - Substitute missing values from mean or median (data imputation) - {inbuilt class in sklearn also}
- embarked means from point they boarded
- It is possible that is does not depend on embarked and name.
- Fare , age will be kept 
- cabin has many NaN values so it can be discarded aslo we can keep a track of person's cbin by the fare price.

In [6]:
columns_to_Drop = ["PassengerId","Name","Ticket","Cabin","Embarked"]

data_clean = data.drop(columns_to_Drop,axis=1)

In [7]:
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500


In [8]:
# One of our class - Sex is in string format
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data_clean["Sex"] = le.fit_transform(data_clean["Sex"])

In [9]:
data_clean.head()
# All males have label 1 and female 0

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [10]:
data_clean.info()
# Fill all age values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


In [10]:
data_clean = data_clean.fillna?
# Fills na with specified values

In [11]:
data_clean = data_clean.fillna(data_clean["Age"].mean())

In [12]:
data_clean.info()
# All have 891 entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
dtypes: float64(2), int32(1), int64(4)
memory usage: 45.3 KB


In [13]:
# Access a particular row
data_clean.loc[2][1]

3.0

In [14]:
data_clean.loc[1]

Survived     1.0000
Pclass       1.0000
Sex          0.0000
Age         38.0000
SibSp        1.0000
Parch        0.0000
Fare        71.2833
Name: 1, dtype: float64

In [15]:
input_col = ["Pclass","Sex","Age","SibSp","Parch","Fare"]
output_col = ["Survived"]

X = data_clean[input_col]
Y = data_clean[output_col]

print(X.shape,Y.shape)

(891, 6) (891, 1)


### Decision Tree Implementation

#### define entropy and information gain

In [16]:
def entropy(col):
    
    counts = np.unique(col,return_counts=True)
    N = float(col.shape[0])
    entropy = 0.0
    
    for i in counts[1]:
        p = i/N
        entropy += (-1.0 * p * np.log2(p))
    
    return entropy

In [17]:
c = np.array([1,1,1,0,0,0])
entropy(c)

1.0

In [18]:
def divide_data(x_data,fkey,fval):
    # work with Pandas dataframe
    # creating 2 empty dataframe
    x_right = pd.DataFrame([],columns=x_data.columns)
    x_left = pd.DataFrame([],columns=x_data.columns)
    
    for i in range(x_data.shape[0]):
        val = x_data[fkey].loc[i]
        if val > fval:
            x_right = x_right.append(x_data.loc[i])
        else:
            x_left = x_left.append(x_data.loc[i])
    
    return x_left,x_right

In [19]:
x_left,x_right = divide_data(data_clean[:10],'Sex',0.5)
print(x_left)
print(x_right)

   Survived  Pclass  Sex   Age  SibSp  Parch     Fare
1       1.0     1.0  0.0  38.0    1.0    0.0  71.2833
2       1.0     3.0  0.0  26.0    0.0    0.0   7.9250
3       1.0     1.0  0.0  35.0    1.0    0.0  53.1000
8       1.0     3.0  0.0  27.0    0.0    2.0  11.1333
9       1.0     2.0  0.0  14.0    1.0    0.0  30.0708
   Survived  Pclass  Sex        Age  SibSp  Parch     Fare
0       0.0     3.0  1.0  22.000000    1.0    0.0   7.2500
4       0.0     3.0  1.0  35.000000    0.0    0.0   8.0500
5       0.0     3.0  1.0  29.699118    0.0    0.0   8.4583
6       0.0     1.0  1.0  54.000000    0.0    0.0  51.8625
7       0.0     3.0  1.0   2.000000    3.0    1.0  21.0750


In [20]:
def information_gain(x_data,fkey,fval):
    # Split data 
    left,right = divide_data(x_data,fkey,fval)
    # we will compute reduction in entropy after this entropy
    # % fo people on left and right
    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]
    
    # All examples can come to one side
    if left.shape[0] == 0 or right.shape[0] == 0:
        return -1000000 # Min information Gain
    
    i_gain = entropy(x_data.Survived) - (l*entropy(left.Survived) + r*entropy(right.Survived))
    return i_gain

In [21]:
# Test Function
for fx in X.columns:
    print(fx)
    print(information_gain(data_clean,fx,data_clean[fx].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606142
Age
0.0008836151229467681
SibSp
0.009584541813400071
Parch
0.015380754493137694
Fare
0.042140692838995464


First node will be Sex as it has maximum ig.

In [29]:
class DecisionTree:
    
    # Constructor
    def __init__(self,depth=0,max_depth=5,n_feat=None):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        self.n_feat = n_feat
    
    def train(self,X_train):
        
        features = ["Pclass","Sex","Age","SibSp","Parch","Fare"]
        if self.n_feat == None:
            self.n_feat = np.random.randint(1,6)
        features =np.random.choice(features,self.n_feat)
        info_gain = []
        
        for i in features:
            i_gain = information_gain(X_train,i,X_train[i].mean())
            info_gain.append(i_gain)
        
        self.fkey = features[np.argmax(info_gain)]
        self.fval = X_train[self.fkey].mean()
        print("Making tree Feature is",self.fkey)
        # Split Data
        data_left,data_right = divide_data(X_train,self.fkey,self.fval)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
        
        # we have reached leaf node
        if data_left.shape[0] == 0 or data_right.shape[0] ==0 :
            if X_train.Survived.mean() >=0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
        
        # Stop early when depth >=maxDepth
        if self.depth >= self.max_depth:
            if X_train.Survived.mean() >=0.5:
                self.target = "Survived"
            else:
                self.target = "Dead"
            return
        # Recursive Case
        self.left = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right = DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        # Setting Target at every node
        if X_train.Survived.mean() >=0.5:
            self.target = "Survived"
        else:
            self.target = "Dead"
        return
    
    def predict(self,test):
        if test[self.fkey]>self.fval:
            # right
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [43]:
def boostraping(X,n_bootstrap):
    index = np.random.randint(low=0, high=len(X), size=n_bootstrap)
    data = X.iloc[index]
    data = data.reset_index(drop=True)
    return data

class Random_Forest:
    def __init__(self,n_trees=5,n_bootstrap=100,max_depth=5,n_feat=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.forest = []
        self.n_bootstrap=n_bootstrap
        self.n_feat = n_feat
        
    def training(self,X):
        self.forest = []
        for i in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth,n_feat=self.n_feat)
            x = boostraping(X,n_bootstrap=self.n_bootstrap)
            tree.train(x)
            self.forest.append(tree)

    def predict(self,X):
        tree_predicted = np.array([tree.predict(X) for tree in self.forest])
        return tree_predicted

#### Some of the features can repeat as they are used in different branches

In [31]:
split = int(0.7*data_clean.shape[0])
train_data = data_clean[:split]
test_data = data_clean[split:]
test_data = test_data.reset_index(drop=True)

In [25]:
print(train_data.shape,test_data.shape)

(623, 7) (268, 7)


In [32]:
y_actual = test_data[output_col]

In [33]:
print(y_actual)

     Survived
0           0
1           0
2           0
3           0
4           1
..        ...
263         0
264         1
265         0
266         1
267         0

[268 rows x 1 columns]


In [44]:
rf_scratch = Random_Forest()

In [45]:
y_actual = test_data[output_col]

In [46]:
rf_scratch.training(train_data)

Making tree Feature is Sex
Making tree Feature is SibSp
Making tree Feature is Sex
Making tree Feature is SibSp
Making tree Feature is Parch
Making tree Feature is Age
Making tree Feature is Pclass
Making tree Feature is Fare
Making tree Feature is Age
Making tree Feature is Pclass
Making tree Feature is Sex
Making tree Feature is Fare
Making tree Feature is Pclass
Making tree Feature is SibSp
Making tree Feature is Sex
Making tree Feature is Age
Making tree Feature is SibSp
Making tree Feature is Sex
Making tree Feature is Pclass
Making tree Feature is Age
Making tree Feature is SibSp
Making tree Feature is Fare
Making tree Feature is Sex
Making tree Feature is Fare
Making tree Feature is Age
Making tree Feature is Fare
Making tree Feature is Pclass
Making tree Feature is Fare
Making tree Feature is Fare
Making tree Feature is Fare
Making tree Feature is Age
Making tree Feature is Parch
Making tree Feature is Fare
Making tree Feature is Fare
Making tree Feature is Age
Making tree Feat

In [47]:
y_pred = []
for i in range(test_data.shape[0]):
    m =np.array(rf_scratch.predict(test_data.loc[i]))
    le = LabelEncoder()
    m = le.fit_transform(m)
    y_pred.append(np.bincount(m).argmax())

In [57]:
np.bincount([1,1,1,1,0,0,0,0,0]).argmax()

0

In [48]:
y_pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [49]:
y_pred = np.array(y_pred)

In [51]:
y_actual = y_actual.values

In [52]:
acc = np.sum((y_pred == y_actual))/y_pred.shape[0]
print(acc)

161.22388059701493
